In [9]:
import pandas as pd
import numpy
import csv
import re


news_P_N = pd.read_csv(r'C:\Users\Desktop\Desktop\주가예측\단순감성분석\title_ratio_pororo_after.txt', delimiter = '\t')
discuss_P_N = pd.read_csv(r'C:\Users\Desktop\Desktop\주가예측\단순감성분석\discuss_title_ratio_pororo_after2.txt', delimiter = '\t')

stock_data = pd.read_csv(r'C:\Users\Desktop\Desktop\주가예측\단순감성분석\주식전처리완료-0라벨제거.csv',encoding='cp949')
stock_data2 = pd.read_csv(r'C:\Users\Desktop\Desktop\주가예측\단순감성분석\주식전처리완료.csv',encoding='cp949')

stock_data=stock_data.drop(columns=['라벨링','기업명','기업코드','시가','고가','저가','종가','거래량'])
stock_data.rename(columns={'날짜':'id'},inplace=True)

stock_data2=stock_data2.drop(columns=['라벨링','기업명','기업코드','시가','고가','저가','거래량'])
stock_data2.rename(columns={'날짜':'id'},inplace=True)

for i in range(len(news_P_N)):
    news_P_N['id'][i] = re.sub("\.","",news_P_N['id'][i])

## train(20170607~20200306 ) 최종 자산이 제일 높을 때를 구함.

max_account=0;
fit_title_num=0;
fit_ratio=0;
fit_discuss_positive=0;

for title_num in range(30):
    for ratio in range(80):
        for discuss_positive in range(20):
            long_id_news = news_P_N[news_P_N['num']>title_num][news_P_N['ratio']>ratio][news_P_N['id']<"20200306"].id

            long_id_news = long_id_news.reset_index(drop=True)

            for i in range(len(long_id_news)):
                long_id_news[i] = re.sub("\.","",long_id_news[i])

            long_id_discuss = discuss_P_N[discuss_P_N['positive']>discuss_positive].id

            long_id_discuss = long_id_discuss.reset_index(drop=True)

            for i in range(len(long_id_discuss)):
                long_id_discuss[i] = re.sub("\.","",long_id_discuss[i])


            real_long_id=[]

            # 위에 해당하는 news의 조건과 종목토론방 조건이 맞는 날에 대해서 뽑음
            for i in long_id_news:
                for k in long_id_discuss:
                    if i==k:
            #             print(i)
                        real_long_id.append(i)

            right = 0
            right_ratio=[]
            false = 0
            false_ratio=[]

            account = 1000000
            init_account = account
            left = 0

            buy = 0
            sell = 0 
            num = 0


            for i in real_long_id:
                for k in range(1000):
                    if(i == str(stock_data2['id'][k])):
                        # 만약에 long을 맞춘 경우
                        if(stock_data2['등락률'][k] >= 0):
                            right+=1
                            right_ratio.append(stock_data2['등락률'][k])
                        # long으로 예상했는데 틀린경우
                        else:
                            false+=1
                            false_ratio.append(stock_data2['등락률'][k])

#                         print("실제 다음날 등락률 : ",stock_data2['등락률'][k])
                        buy = stock_data2['종가'][k]*(account//stock_data2['종가'][k])
                        num = (account//stock_data2['종가'][k])
#                         print("구매 수:",num)
                        # 다음날이 -1 임!! 
                        sell = stock_data2['종가'][k-1]*num
#                         print("구매가격 : ",stock_data2['종가'][k])
#                         print("구매가치 : ",buy)
                        left = account - buy
#                         print("잔고--   : ",left)
#                         print("판매가격 : ",stock_data2['종가'][k-1])
#                         print("판매가치 : ",sell)

                        account = sell + left
#                         print("현재 자산 : ",account)

#             print(right_ratio)
#             print(false_ratio)
            print("기사수 : ", title_num)
            print("기사긍정비율 : ",ratio)
            print("긍정토론수 : ",discuss_positive)
            
            if((right + false)!=0):
                print("acc = ",right/(right + false) * 100 ,"%")
                print("최종 account : ",account)
                
            if(account >= init_account):
                loss = account - init_account
                loss = 1/loss
                print("loss = ",loss)
            else :
                loss = init_account - account
                loss = 1/loss
                print("loss = ",loss)
                
            print("현재 갯수 : ",(right + false))
            
            # 예측이 5개 이상인 경우만 판단
            if (account > max_account and (right + false) >= 5 ):
                max_account = account
                fit_title_num=title_num;
                fit_ratio=ratio;
                fit_discuss_positive=discuss_positive;
            
            print("현재 max : ",max_account)
                
                

                
# 가장 높게 나온 최종 값과 해당 임계값들 출력
print("가장 최적 account : ",max_account)
print("가장 최적 조건 : ",fit_title_num,fit_ratio,fit_discuss_positive)

<ipython-input-9-673814d5c8bc>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_P_N['id'][i] = re.sub("\.","",news_P_N['id'][i])
<ipython-input-9-673814d5c8bc>:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  long_id_news = news_P_N[news_P_N['num']>title_num][news_P_N['ratio']>ratio][news_P_N['id']<"20200306"].id


기사수 :  0
기사긍정비율 :  0
긍정토론수 :  0
acc =  48.0 %
최종 account :  682400
loss =  3.1486146095717885e-06
현재 갯수 :  475
현재 max :  682400


<ipython-input-9-673814d5c8bc>:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  long_id_news = news_P_N[news_P_N['num']>title_num][news_P_N['ratio']>ratio][news_P_N['id']<"20200306"].id


기사수 :  0
기사긍정비율 :  0
긍정토론수 :  1
acc =  48.4641638225256 %
최종 account :  699000
loss =  3.3222591362126247e-06
현재 갯수 :  293
현재 max :  699000


<ipython-input-9-673814d5c8bc>:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  long_id_news = news_P_N[news_P_N['num']>title_num][news_P_N['ratio']>ratio][news_P_N['id']<"20200306"].id


...

기사수 :  0
기사긍정비율 :  3
긍정토론수 :  1
acc =  48.4641638225256 %
최종 account :  699000
loss =  3.3222591362126247e-06
현재 갯수 :  293
현재 max :  1220700


<ipython-input-9-673814d5c8bc>:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  long_id_news = news_P_N[news_P_N['num']>title_num][news_P_N['ratio']>ratio][news_P_N['id']<"20200306"].id


기사수 :  0
기사긍정비율 :  3
긍정토론수 :  2
acc =  50.0 %
최종 account :  741400
loss =  3.8669760247486465e-06
현재 갯수 :  178
현재 max :  1220700


<ipython-input-9-673814d5c8bc>:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  long_id_news = news_P_N[news_P_N['num']>title_num][news_P_N['ratio']>ratio][news_P_N['id']<"20200306"].id


KeyboardInterrupt: 

In [ ]:
## test 20200710 ~ 20210510
## 10,43,3 test에서의 결과
long_id_news = news_P_N[news_P_N['num']>8][news_P_N['ratio']>45][news_P_N['id']>="20200710"].id

long_id_news = long_id_news.reset_index(drop=True)

for i in range(len(long_id_news)):
    long_id_news[i] = re.sub("\.","",long_id_news[i])

long_id_discuss = discuss_P_N[discuss_P_N['positive']>3].id

long_id_discuss = long_id_discuss.reset_index(drop=True)

for i in range(len(long_id_discuss)):
    long_id_discuss[i] = re.sub("\.","",long_id_discuss[i])
    
    
real_long_id=[]

for i in long_id_news:
    for k in long_id_discuss:
        if i==k:
#             print(i)
            real_long_id.append(i)

right = 0
right_ratio=[]
false = 0
false_ratio=[]

date=[]
all_ratio=[]

account = 1000000
left = 0

buy = 0
sell = 0 
num = 0

result={}


for i in real_long_id:
    for k in range(1000):
        if(i == str(stock_data2['id'][k])):
            if(stock_data2['등락률'][k] >= 0):
                right+=1
                right_ratio.append(stock_data2['등락률'][k])
            else:
                false+=1
                false_ratio.append(stock_data2['등락률'][k])
            
            date.append(stock_data2['id'][k])
            all_ratio.append(stock_data2['등락률'][k])
            
            result= {"id" : date , "ratio":all_ratio }  
            df = pd.DataFrame(result)  #df로 변환
            
            # 인덱스 없애기
            df.to_csv('last_result.txt', sep = '\t', index = False)
            
            print("실제 다음날 등락률 : ",stock_data2['등락률'][k])
            buy = stock_data2['종가'][k]*(account//stock_data2['종가'][k])
            num = (account//stock_data2['종가'][k])
            print("구매 수:",num)
            # 다음날이 -1 임!! 
            sell = stock_data2['종가'][k-1]*num
            print("구매가격 : ",stock_data2['종가'][k])
            print("구매가치 : ",buy)
            left = account - buy
            print("잔고--   : ",left)
            print("판매가격 : ",stock_data2['종가'][k-1])
            print("판매가치 : ",sell)
            
            account = sell + left
            print("현재 자산 : ",account)
            
            print("현재 갯수 : ",(right + false))
                
print(right_ratio)
print(false_ratio)
# print(all_check)
print("acc = ",right/(right + false) * 100 ,"%")
print("최종 account : ",account)


In [2]:
!pip install wandb -qqq
import wandb
wandb.login()

wandb: Currently logged in as: yeongu (use `wandb login --relogin` to force relogin)


True

In [3]:
import math
import random
import pandas as pd
import numpy
import csv
import re
import time

!wandb agent yeongu/train2/zv95s4xd

news_P_N = pd.read_csv(r'C:\Users\Desktop\Desktop\주가예측\단순감성분석\title_ratio_pororo_after.txt', delimiter = '\t')
discuss_P_N = pd.read_csv(r'C:\Users\Desktop\Desktop\주가예측\단순감성분석\discuss_title_ratio_pororo_after2.txt', delimiter = '\t')

stock_data = pd.read_csv(r'C:\Users\Desktop\Desktop\주가예측\단순감성분석\주식전처리완료-0라벨제거.csv',encoding='cp949')
stock_data2 = pd.read_csv(r'C:\Users\Desktop\Desktop\주가예측\단순감성분석\주식전처리완료.csv',encoding='cp949')

stock_data=stock_data.drop(columns=['라벨링','기업명','기업코드','시가','고가','저가','종가','거래량'])
stock_data.rename(columns={'날짜':'id'},inplace=True)

stock_data2=stock_data2.drop(columns=['라벨링','기업명','기업코드','시가','고가','저가','거래량'])
stock_data2.rename(columns={'날짜':'id'},inplace=True)

for i in range(len(news_P_N)):
    news_P_N['id'][i] = re.sub("\.","",news_P_N['id'][i])

# Start a new run, tracking hyperparameters in config
wandb.init(project="train2", config={
    "title_num": 0,
    "ratio": 0,
    "discuss_positive": 0,
})
config = wandb.config


for step in range(10):
    long_id_news = news_P_N[news_P_N['num']>config.title_num][news_P_N['ratio']>config.ratio][news_P_N['id']<"20200306"].id

    long_id_news = long_id_news.reset_index(drop=True)

    for i in range(len(long_id_news)):
        long_id_news[i] = re.sub("\.","",long_id_news[i])

    long_id_discuss = discuss_P_N[discuss_P_N['positive']>config.discuss_positive].id

    long_id_discuss = long_id_discuss.reset_index(drop=True)

    for i in range(len(long_id_discuss)):
        long_id_discuss[i] = re.sub("\.","",long_id_discuss[i])


    real_long_id=[]

    # 위에 해당하는 news의 조건과 종목토론방 조건이 맞는 날에 대해서 뽑음
    for i in long_id_news:
        for k in long_id_discuss:
            if i==k:
    #             print(i)
                real_long_id.append(i)

    right = 0
    right_ratio=[]
    false = 0
    false_ratio=[]

    account = 1000000
    init_account = account
    left = 0

    buy = 0
    sell = 0 
    num = 0


    for i in real_long_id:
        for k in range(1000):
            if(i == str(stock_data2['id'][k])):
                # 만약에 long을 맞춘 경우
                if(stock_data2['등락률'][k] >= 0):
                    right+=1
                    right_ratio.append(stock_data2['등락률'][k])
                # long으로 예상했는데 틀린경우
                else:
                    false+=1
                    false_ratio.append(stock_data2['등락률'][k])

    #                         print("실제 다음날 등락률 : ",stock_data2['등락률'][k])
                buy = stock_data2['종가'][k]*(account//stock_data2['종가'][k])
                num = (account//stock_data2['종가'][k])
    #                         print("구매 수:",num)
                # 다음날이 -1 임!! 
                sell = stock_data2['종가'][k-1]*num
    #                         print("구매가격 : ",stock_data2['종가'][k])
    #                         print("구매가치 : ",buy)
                left = account - buy
    #                         print("잔고--   : ",left)
    #                         print("판매가격 : ",stock_data2['종가'][k-1])
    #                         print("판매가치 : ",sell)

                account = sell + left
    #                         print("현재 자산 : ",account)
                wandb.log({"account":account})
                time.sleep(1)
    

if wandb.run is not None:
      wandb.finish()
        

wandb: Starting wandb agent \U0001f575\ufe0f
2021-06-28 11:45:47,148 - wandb.wandb_agent - INFO - Running runs: []
2021-06-28 11:45:47,452 - wandb.wandb_agent - INFO - Agent received command: run
2021-06-28 11:45:47,453 - wandb.wandb_agent - INFO - Agent starting run with config:
	discuss_positive: 2
	ratio: 64
	title_num: 26
2021-06-28 11:45:47,454 - wandb.wandb_agent - INFO - About to run command: python train2.py --discuss_positive=2 --ratio=64 --title_num=26
python: can't open file 'train2.py': [Errno 2] No such file or directory
2021-06-28 11:45:52,473 - wandb.wandb_agent - INFO - Running runs: ['cff1bg4l']
2021-06-28 11:45:52,473 - wandb.wandb_agent - INFO - Cleaning up finished run: cff1bg4l
2021-06-28 11:45:53,099 - wandb.wandb_agent - INFO - Agent received command: run
2021-06-28 11:45:53,099 - wandb.wandb_agent - INFO - Agent starting run with config:
	discuss_positive: 18
	ratio: 38
	title_num: 29
2021-06-28 11:45:53,101 - wandb.wandb_agent - INFO - About to run command: pyt

<ipython-input-3-4479aa72112f>:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  long_id_news = news_P_N[news_P_N['num']>config.title_num][news_P_N['ratio']>config.ratio][news_P_N['id']<"20200306"].id
<ipython-input-3-4479aa72112f>:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  long_id_news = news_P_N[news_P_N['num']>config.title_num][news_P_N['ratio']>config.ratio][news_P_N['id']<"20200306"].id
<ipython-input-3-4479aa72112f>:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  long_id_news = news_P_N[news_P_N['num']>config.title_num][news_P_N['ratio']>config.ratio][news_P_N['id']<"20200306"].id


KeyboardInterrupt: 

In [6]:
sweep_config = {
    'method': 'bayes'
}

metric={
      'goal': 'maximize',
      'name': 'account'
}
    
sweep_config['metric'] = metric

parameters_dict = {
}

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'discuss_positive': {
        'max': 8, # 1 ~ 갯수의 평균x2 까지
        'min': 1
    },
      'ratio': {
        'max': 72, # 평균 비율 ~ X2 까지
        'min': 36
      },
      'title_num': {
        'max': 14 , # 1 ~ 평균 까지
        'min': 1
      }
})

import pprint

pprint.pprint(sweep_config)



{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'account'},
 'parameters': {'discuss_positive': {'max': 8, 'min': 1},
                'ratio': {'max': 72, 'min': 36},
                'title_num': {'max': 14, 'min': 1}}}


In [40]:
import pandas as pd
import numpy
import csv
!pip install wandb -qqq
import wandb
import pprint
import re


stock =["유한양행","씨젠","오스템임플란트","차바이오텍","헬릭스미스","메지온","에이치엘비생명과학","대웅","녹십자홀딩스",
"동국제약","CMG제약","영진약품","메디포스트","코미팜","엔케이맥스","바디텍메드","JW중외제약","클래시스","종근당홀딩스",
"서흥","카이노스메드","앱클론","아이센스","에이프로젠제약","바텍","삼진제약","동화약품","수젠텍","제일약품",
"뷰웍스","테라젠이텍스","한독","케어젠","대원제약","유비케어","일동제약","동성제약","지노믹트리","제일파마홀딩스","휴마시스","환인제약","팜젠사이언스","진원생명과학","종근당바이오","종근당","제넥신","일양약품","이연제약","유나이티드제약","오리엔트바이오","셀트리온제약","삼천당제약","삼성제약","삼성바이오로직스","부광약품","명문제약","대웅제약","국제약품","광동제약","고려제약","경보제약",
     "메디톡스","아미코젠","인트론바이오","알테오젠","한미사이언스","바이오니아","레고켐바이오","셀트리온","셀트리온헬스케어","안트로젠",
"유바이오로직스","보령제약","파마리서치","엔지켐생명과학","KPX생명과학","에스티팜","휴온스","한미약품","한올바이오파마","지트리비앤티","녹십자랩셀","올릭스",
"크리스탈지노믹스","휴젤","셀리버리","이수앱지스","바이넥스","휴메딕스","휴온스글로벌"]


i="유한양행"


def train(config=None):
    
    
    j="유한양행"
    
    news_P_N = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\제약테마_감성분석완료\title_ratio_all\title_ratio_pororo_after_{j}.txt', delimiter = '\t')
    discuss_P_N = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\제약테마_감성분석완료\discuss_ratio_all\discuss_title_ratio_pororo_after2_{j}.txt', delimiter = '\t')


    
    stock_data2 = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\주식데이터\{j}_일봉_train.csv',encoding='cp949')

    stock_data2=stock_data2.drop(columns=['기업명','기업코드','시간','시가','고가','저가','거래량'])
    stock_data2.rename(columns={'날짜':'id'},inplace=True)
    

    for i in range(len(news_P_N)):
        news_P_N['id'][i] = re.sub("\.","",news_P_N['id'][i])

    # Start a new run, tracking hyperparameters in config
    with wandb.init(config=config):
        config=wandb.config
        
        long_id_news = news_P_N[news_P_N['num']>=config.title_num][news_P_N['ratio']>=config.ratio].id

        long_id_news = long_id_news.reset_index(drop=True)

        for i in range(len(long_id_news)):
            long_id_news[i] = re.sub("\.","",long_id_news[i])

        long_id_discuss = discuss_P_N[discuss_P_N['positive']>=config.discuss_positive].id

        long_id_discuss = long_id_discuss.reset_index(drop=True)

        for i in range(len(long_id_discuss)):
            long_id_discuss[i] = re.sub("\.","",long_id_discuss[i])


        real_long_id=[]

        # 위에 해당하는 news의 조건과 종목토론방 조건이 맞는 날에 대해서 뽑음
        for i in long_id_news:
            for k in long_id_discuss:
                if i==k:
        #             print(i)
                    real_long_id.append(i)

        right = 0
        right_ratio=[]
        false = 0
        false_ratio=[]

        account = 10000000
        init_account = account
        left = 0

        buy = 0
        sell = 0 
        num = 0


        for i in real_long_id:
            for k in range(len(stock_data2)):
                if(i == str(stock_data2['id'][k])):
                    # 만약에 long을 맞춘 경우
                    if(stock_data2['등락률'][k] >= 0):
                        right+=1
                        right_ratio.append(stock_data2['등락률'][k])
                    # long으로 예상했는데 틀린경우
                    else:
                        false+=1
                        false_ratio.append(stock_data2['등락률'][k])

                    print("======================")
                    print("실제 다음날 등락률 : ",stock_data2['등락률'][k])
                    print(stock_data2["id"][k])
#                     buy = (stock_data2['종가'][k] * 1.00015) * ( account//(stock_data2['종가'][k] * 1.00015) )
                    num = (account//(stock_data2['종가'][k] * 1.00015))
                    left = account - num * (stock_data2['종가'][k] * 1.00015)
                    print("구매가 : ",stock_data2['종가'][k] * 1.00015)
                    print("구매 수:",num)
                    print("잔고   : ",left)
            
                    # 다음날이 -1 임!! 
                    sell = stock_data2['종가'][k-1]*num

                    print("판매가격 : ",stock_data2['종가'][k-1])
                    print("판매가치 : ",sell)

                    account = sell * 0.9935 + left
                    acc = (right/(right + false)) * 100
                    num = (right + false)
                    print("현재 자산 : ",account)
                    print("======================")
                
           
        print(j,"최종자산 : ",account," acc : ",acc," num :",num)
        wandb.log({"account": account, "acc": acc, "num": num})
        

news_P_N = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\제약테마_감성분석완료\title_ratio_all\title_ratio_pororo_after_{i}.txt', delimiter = '\t')
discuss_P_N = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\제약테마_감성분석완료\discuss_ratio_all\discuss_title_ratio_pororo_after2_{i}.txt', delimiter = '\t')

sweep_config = {
    'method': 'bayes'
}

metric={
      'goal': 'maximize',
      'name': 'account'
}

sweep_config['metric'] = metric

parameters_dict = {
}

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'discuss_positive': {
        'max': round(news_P_N.describe()['positive'][1]) * 2, # 1 ~ 갯수의 평균x2 까지
        'min': 1
    },
      'ratio': {
        'max': round(news_P_N.describe()['ratio'][1]) * 2, # 평균 비율 ~ X2 까지
        'min': round(news_P_N.describe()['ratio'][1])
      },
      'title_num': {
        'max': round(news_P_N.describe()['num'][1]) , # 1 ~ 평균 까지
        'min': 1
      }
})

pprint.pprint(sweep_config)

sweep_id = wandb.sweep(sweep_config, project=f"감성분석_{i}")            
wandb.agent(sweep_id, train, count=200)


{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'account'},
 'parameters': {'discuss_positive': {'max': 12, 'min': 1},
                'ratio': {'max': 80, 'min': 40},
                'title_num': {'max': 15, 'min': 1}}}
Create sweep with ID: khrw9f6a
Sweep URL: https://wandb.ai/yeongu/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_%EC%9C%A0%ED%95%9C%EC%96%91%ED%96%89/sweeps/khrw9f6a


wandb: Agent Starting Run: sa2a70l9 with config:
wandb: 	discuss_positive: 4
wandb: 	ratio: 64
wandb: 	title_num: 5
<ipython-input-40-ffc3288b7101>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_P_N['id'][i] = re.sub("\.","",news_P_N['id'][i])


<ipython-input-40-ffc3288b7101>:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  long_id_news = news_P_N[news_P_N['num']>=config.title_num][news_P_N['ratio']>=config.ratio].id


실제 다음날 등락률 :  -2.128640776699029
20181204
구매가 :  41206.18
구매 수: 242.0
잔고   :  28104.43999999948
판매가격 :  40323
판매가치 :  9758166.0
현재 자산 :  9722842.361
실제 다음날 등락률 :  -0.8187186828919113
20190214
구매가 :  44710.7056
구매 수: 217.0
잔고   :  20619.245799999684
판매가격 :  44338
판매가치 :  9621346.0
현재 자산 :  9579426.4968
유한양행 최종자산 :  9579426.4968  acc :  0.0  num : 2


account,9579426.4968
acc,0.0
num,2
_runtime,7
_timestamp,1625813963
_step,0


...

account,8044757.74985
acc,47.36842
num,38
_runtime,8
_timestamp,1625818168
_step,0


account,▁
acc,▁
num,▁
_runtime,▁
_timestamp,▁
_step,▁


429 response executing GraphQL.
{"error":"rate limit exceeded"}

429 response executing GraphQL.
{"error":"rate limit exceeded"}

Retry attempt failed:
Traceback (most recent call last):
  File "C:\Users\Desktop\AppData\Roaming\Python\Python38\site-packages\wandb\sdk\lib\retry.py", line 102, in __call__
    result = self._call_fn(*args, **kwargs)
  File "C:\Users\Desktop\AppData\Roaming\Python\Python38\site-packages\wandb\sdk\internal\internal_api.py", line 133, in execute
    six.reraise(*sys.exc_info())
  File "C:\ProgramData\Anaconda3\lib\site-packages\six.py", line 703, in reraise
    raise value
  File "C:\Users\Desktop\AppData\Roaming\Python\Python38\site-packages\wandb\sdk\internal\internal_api.py", line 127, in execute
    return self.client.execute(*args, **kwargs)
  File "C:\Users\Desktop\AppData\Roaming\Python\Python38\site-packages\wandb\vendor\gql-0.2.0\gql\client.py", line 52, in execute
    result = self._get_result(document, *args, **kwargs)
  File "C:\Users\Desktop\App

<ipython-input-40-ffc3288b7101>:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  long_id_news = news_P_N[news_P_N['num']>=config.title_num][news_P_N['ratio']>=config.ratio].id
429 response executing GraphQL.
{"error":"rate limit exceeded"}

Retry attempt failed:
Traceback (most recent call last):
  File "C:\Users\Desktop\AppData\Roaming\Python\Python38\site-packages\wandb\sdk\lib\retry.py", line 102, in __call__
    result = self._call_fn(*args, **kwargs)
  File "C:\Users\Desktop\AppData\Roaming\Python\Python38\site-packages\wandb\sdk\internal\internal_api.py", line 133, in execute
    six.reraise(*sys.exc_info())
  File "C:\ProgramData\Anaconda3\lib\site-packages\six.py", line 703, in reraise
    raise value
  File "C:\Users\Desktop\AppData\Roaming\Python\Python38\site-packages\wandb\sdk\internal\internal_api.py", line 127, in execute
    return self.client.execute(*args, **kwargs)
  File "C:\Users\Desktop\AppData\Roaming\Python\Python38\site-packages\

Run kbsi10vf errored: UnboundLocalError("local variable 'acc' referenced before assignment")
wandb: ERROR Run kbsi10vf errored: UnboundLocalError("local variable 'acc' referenced before assignment")


In [ ]:
news_P_N = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\제약테마_감성분석완료\title_ratio_all/title_ratio_pororo_after_{j}.txt', delimiter = '\t')
discuss_P_N = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\제약테마_감성분석완료\discuss_ratio_all/discuss_title_ratio_pororo_after2_{j}.txt', delimiter = '\t')



stock_data2 = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\주식데이터/{j}_일봉_test.csv',encoding='cp949')

stock_data2=stock_data2.drop(columns=['기업명','기업코드','시간','시가','고가','저가','거래량'])
stock_data2.rename(columns={'날짜':'id'},inplace=True)


for i in range(len(news_P_N)):
    news_P_N['id'][i] = re.sub("\.","",news_P_N['id'][i])

long_id_news = news_P_N[news_P_N['num']>7[news_P_N['ratio']>thre[thre_num+1]][news_P_N['id']>="20200710"].id

long_id_news = long_id_news.reset_index(drop=True)

for i in range(len(long_id_news)):
    long_id_news[i] = re.sub("\.","",long_id_news[i])

long_id_discuss = discuss_P_N[discuss_P_N['positive']>1].id


long_id_discuss = long_id_discuss.reset_index(drop=True)

for i in range(len(long_id_discuss)):
    long_id_discuss[i] = re.sub("\.","",long_id_discuss[i])


real_long_id=[]

for i in long_id_news:
    for k in long_id_discuss:
        if i==k:
#             print(i)
            real_long_id.append(i)

right = 0
right_ratio=[]
false = 0
false_ratio=[]

date=[]
all_ratio=[]

account = 10000000
left = 0

buy = 0
sell = 0 
num = 0

result={}


for i in real_long_id:
    for k in range(len(stock_data2)):
        if(i == str(stock_data2['id'][k]) and stock_data2['id'][k]!=20210617):
            if(stock_data2['등락률'][k] > 0):
                right+=1
                right_ratio.append(stock_data2['등락률'][k])
            else:
                false+=1
                false_ratio.append(stock_data2['등락률'][k])

            date.append(stock_data2['id'][k])
            all_ratio.append(stock_data2['등락률'][k])


            print("======================")
            print("실제 다음날 등락률 : ",stock_data2['등락률'][k])
            buy = stock_data2['종가'][k]*(account//(stock_data2['종가'][k]* 1.00015))
            num = (account//(stock_data2['종가'][k]* 1.00015))
            print("구매 수:",num)
            # 다음날이 -1 임!! 
            sell = stock_data2['종가'][k-1]*num
            print("구매가격 : ",stock_data2['종가'][k] * 1.00015)
            print("구매가치 : ",buy)
            left = account - buy
            print("잔고--   : ",left)
            print("판매가격 : ",stock_data2['종가'][k-1])
            print("판매가치 : ",sell)

            account = sell * 0.9935 + left
            print("현재 자산 : ",account)

            print("현재 갯수 : ",(right + false))
            print("======================")


print(right_ratio)
print(false_ratio)
# print(all_check)
if(right + false !=0):
    print("acc = ",right/(right + false) * 100 ,"%")

print(j," 최종 account : ",account)

In [78]:
## test 20200710 ~ 20210510
## 10,43,3 test에서의 결과

stock =["유한양행","씨젠","오스템임플란트","차바이오텍","헬릭스미스","메지온","에이치엘비생명과학","대웅","녹십자홀딩스",
"동국제약","CMG제약","영진약품","메디포스트","코미팜","엔케이맥스","바디텍메드","JW중외제약","클래시스","종근당홀딩스",
"서흥","카이노스메드","앱클론","아이센스","에이프로젠제약","바텍","삼진제약","동화약품","수젠텍","제일약품",
"뷰웍스","테라젠이텍스","한독","케어젠","대원제약","유비케어","일동제약","동성제약","지노믹트리","제일파마홀딩스","휴마시스",
        "환인제약","진원생명과학","종근당바이오","종근당","제넥신","일양약품","이연제약","유나이티드제약","오리엔트바이오"
        ,"셀트리온제약","삼천당제약","삼성제약","삼성바이오로직스","부광약품","명문제약","대웅제약","국제약품","광동제약"
        ,"고려제약","경보제약",
     "메디톡스","아미코젠","인트론바이오","알테오젠","한미사이언스","바이오니아","레고켐바이오","셀트리온","셀트리온헬스케어","안트로젠",
"유바이오로직스","보령제약","파마리서치","엔지켐생명과학","KPX생명과학","에스티팜","휴온스","한미약품","한올바이오파마","지트리비앤티",
        "녹십자랩셀","올릭스",
"크리스탈지노믹스","휴젤","셀리버리","이수앱지스","바이넥스","휴메딕스","휴온스글로벌"]

thre = [12,46,14,
        1,60,7,
        3,88,4,
        1,60,5,
        7,63,4,
        4,59,7,
        1,55,6,
        6,31,14,
        2,61,3,
        3,47,5,
        2,28,4,
        2,58,4,
        4,70,2,
        4,32,1,
        2,63,2,
        4,76,4,
        4,62,6,
       3,45,1,
       2,62,3,
       1,73,2,
       2,29,2,
       4,52,4,
       2,63,3,
       2,51,4,
       3,71,2,
       4,54,4,
       2,53,2,
       1,72,8,
       1,43,4,
       2,48,3,
       4,77,1,
       7,62,4,
       2,47,2,
       3,67,4,
       4,76,3,
       2,65,4,
       4,70,5,
       1,54,1,
       1,36,1,
       4,64,1,
       2,63,2,
       4,37,8,
       4,56,4,
       3,46,17,
       6,59,8,
       4,34,6,
       1,41,3,
       1,52,5,
       1,49,3,
       10,59,12,
       1,37,5,
       1,50,4,
       1,66,1,
       4,64,4,
       1,84,4,
       6,48,5,
       1,56,4,
       3,60,6,
       3,70,4,
       2,78,3,
       12,39,1,
       2,76,3,
       3,74,4,
       8,62,4,
       2,50,4,
       4,89,4,
       4,67,1,
       62,69,59,
       16,63,29,
       2,58,2,
       2,62,2,
       5,76,6,
       2,55,3,
       4,42,4,
       4,72,4,
       1,60,4,
       7,35,9,
       14,42,10,
       4,53,6,
       1,56,4,
       4,44,1,
       2,74,2,
       1,38,2,
       1,81,13,
       4,38,5,
       2,59,5,
       4,56,5,
       2,79,3,
       1,39,4]

thre_num=0

for j in stock:
    news_P_N = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\제약테마_감성분석완료\title_ratio_all/title_ratio_pororo_after_{j}.txt', delimiter = '\t')
    discuss_P_N = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\제약테마_감성분석완료\discuss_ratio_all/discuss_title_ratio_pororo_after2_{j}.txt', delimiter = '\t')



    stock_data2 = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\주식데이터/{j}_일봉_test.csv',encoding='cp949')

    stock_data2=stock_data2.drop(columns=['기업명','기업코드','시간','시가','고가','저가','거래량'])
    stock_data2.rename(columns={'날짜':'id'},inplace=True)


    for i in range(len(news_P_N)):
        news_P_N['id'][i] = re.sub("\.","",news_P_N['id'][i])

    long_id_news = news_P_N[news_P_N['num']>=thre[thre_num+2]][news_P_N['ratio']>=thre[thre_num+1]][news_P_N['id']>="20200710"].id

    long_id_news = long_id_news.reset_index(drop=True)

    for i in range(len(long_id_news)):
        long_id_news[i] = re.sub("\.","",long_id_news[i])

    long_id_discuss = discuss_P_N[discuss_P_N['positive']>=thre[thre_num]].id
    

    long_id_discuss = long_id_discuss.reset_index(drop=True)

    for i in range(len(long_id_discuss)):
        long_id_discuss[i] = re.sub("\.","",long_id_discuss[i])


    real_long_id=[]

    for i in long_id_news:
        for k in long_id_discuss:
            if i==k:
    #             print(i)
                real_long_id.append(i)

    right = 0
    right_ratio=[]
    false = 0
    false_ratio=[]

    date=[]
    all_ratio=[]

    account = 10000000
    left = 0

    buy = 0
    sell = 0 
    num = 0

    result={}


    for i in real_long_id:
        for k in range(len(stock_data2)):
            if(i == str(stock_data2['id'][k]) and stock_data2['id'][k]!=20210617):
                if(stock_data2['등락률'][k] > 0):
                    right+=1
                    right_ratio.append(stock_data2['등락률'][k])
                else:
                    false+=1
                    false_ratio.append(stock_data2['등락률'][k])

                date.append(stock_data2['id'][k])
                all_ratio.append(stock_data2['등락률'][k])


                print("======================")
                print(stock_data2['id'][k])
                print("실제 다음날 등락률 : ",stock_data2['등락률'][k])
                buy = stock_data2['종가'][k]*(account//(stock_data2['종가'][k]* 1.00015))
                num = (account//(stock_data2['종가'][k]* 1.00015))
                print("구매 수:",num)
                # 다음날이 -1 임!! 
                sell = stock_data2['종가'][k-1]*num
                print("구매가격 : ",stock_data2['종가'][k] * 1.00015)
                print("구매가치 : ",buy)
                left = account - buy
                print("잔고--   : ",left)
                print("판매가격 : ",stock_data2['종가'][k-1])
                print("판매가치 : ",sell)

                account = sell * 0.9935 + left
                print("현재 자산 : ",account)

                print("현재 갯수 : ",(right + false))
                print("======================")
                
                
                result= {"name" : j,"id" : date , "ratio":all_ratio  , "account":account }  
                df = pd.DataFrame(result)  #df로 변환


    print(right_ratio)
    print(false_ratio)
    # print(all_check)
    if(right + false !=0):
        print("acc = ",right/(right + false) * 100 ,"%")
    else:
        thre_num=thre_num+3
        continue
    print(j," 최종 account : ",account)
    print(j," diss : ",thre[thre_num], " ratio : ",thre[thre_num+1], " title : ",thre[thre_num+2])
    thre_num=thre_num+3
    
    df.to_csv('last_result2.csv',  index = False, mode='a',header=False, encoding = 'euc-kr')
       

<ipython-input-78-7d1a317e7708>:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_P_N['id'][i] = re.sub("\.","",news_P_N['id'][i])
<ipython-input-78-7d1a317e7708>:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  long_id_news = news_P_N[news_P_N['num']>=thre[thre_num+2]][news_P_N['ratio']>=thre[thre_num+1]][news_P_N['id']>="20200710"].id


20200713
실제 다음날 등락률 :  0.9757786876897064
구매 수: 203.0
구매가격 :  49096.36335000001
구매가치 :  9965067.0
잔고--   :  34933.0
판매가격 :  49568
판매가치 :  10062304.0
현재 자산 :  10031832.024
현재 갯수 :  1
20200714
실제 다음날 등락률 :  -0.7726759199483538
구매 수: 202.0
구매가격 :  49575.43520000001
구매가치 :  10012736.0
잔고--   :  19096.02400000021
판매가격 :  49185
판매가치 :  9935370.0
현재 자산 :  9889886.119
현재 갯수 :  2
20200722
실제 다음날 등락률 :  1.5227517018989611
구매 수: 196.0
구매가격 :  50245.53570000001
구매가치 :  9846648.0
잔고--   :  43238.11900000088
판매가격 :  51003
판매가치 :  9996588.0
현재 자산 :  9974848.297000002
현재 갯수 :  3
20200723
실제 다음날 등락률 :  6.003568417544066
구매 수: 195.0
구매가격 :  51010.65045000001
구매가치 :  9945585.0
잔고--   :  29263.297000002116
판매가격 :  54065
판매가치 :  10542675.0
현재 자산 :  10503410.909500003
현재 갯수 :  4
20200806
실제 다음날 등락률 :  1.0592550433187884
구매 수: 166.0
구매가격 :  63261.4878
구매가치 :  10499832.0
잔고--   :  3578.909500002861
판매가격 :  63922
판매가치 :  10611052.0
현재 자산 :  10545659.071500003
현재 갯수 :  5
20200825
실제 다음날 등락률 :  -1.33102478086787

PermissionError: [Errno 13] Permission denied: 'last_result2.csv'

In [126]:
## test 20200710 ~ 20210510
## 10,43,3 test에서의 결과

## 하나하나 각각 테스트

stock =["유한양행","씨젠","오스템임플란트","차바이오텍","헬릭스미스","메지온","에이치엘비생명과학","대웅","녹십자홀딩스",
"동국제약","CMG제약","영진약품","메디포스트","코미팜","엔케이맥스","바디텍메드","JW중외제약","클래시스","종근당홀딩스",
"서흥","카이노스메드","앱클론","아이센스","에이프로젠제약","바텍","삼진제약","동화약품","수젠텍","제일약품",
"뷰웍스","테라젠이텍스","한독","케어젠","대원제약","유비케어","일동제약","동성제약","지노믹트리","제일파마홀딩스","휴마시스",
        "환인제약","진원생명과학","종근당바이오","종근당","제넥신","일양약품","이연제약","유나이티드제약","오리엔트바이오"
        ,"셀트리온제약","삼천당제약","삼성제약","삼성바이오로직스","부광약품","명문제약","대웅제약","국제약품","광동제약"
        ,"고려제약","경보제약",
     "메디톡스","아미코젠","인트론바이오","알테오젠","한미사이언스","바이오니아","레고켐바이오","셀트리온","셀트리온헬스케어","안트로젠",
"유바이오로직스","보령제약","파마리서치","엔지켐생명과학","KPX생명과학","에스티팜","휴온스","한미약품","한올바이오파마","지트리비앤티",
        "녹십자랩셀","올릭스",
"크리스탈지노믹스","휴젤","셀리버리","이수앱지스","바이넥스","휴메딕스","휴온스글로벌"]


i="유나이티드제약"

news_P_N = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\제약테마_감성분석완료\title_ratio_all/title_ratio_pororo_after_{i}.txt', delimiter = '\t')
discuss_P_N = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\제약테마_감성분석완료\discuss_ratio_all/discuss_title_ratio_pororo_after2_{i}.txt', delimiter = '\t')



stock_data2 = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\주식데이터/{i}_일봉_test.csv',encoding='cp949')

stock_data2=stock_data2.drop(columns=['기업명','기업코드','시간','시가','고가','저가','거래량'])
stock_data2.rename(columns={'날짜':'id'},inplace=True)


for i in range(len(news_P_N)):
    news_P_N['id'][i] = re.sub("\.","",news_P_N['id'][i])

long_id_news = news_P_N[news_P_N['num']>=7][news_P_N['ratio']>=16][news_P_N['id']>="20200710"].id

long_id_news = long_id_news.reset_index(drop=True)

print(long_id_news)

for i in range(len(long_id_news)):
    long_id_news[i] = re.sub("\.","",long_id_news[i])

long_id_discuss = discuss_P_N[discuss_P_N['positive']>=1].id


long_id_discuss = long_id_discuss.reset_index(drop=True)

for i in range(len(long_id_discuss)):
    long_id_discuss[i] = re.sub("\.","",long_id_discuss[i])


real_long_id=[]

for i in long_id_news:
    for k in long_id_discuss:
        if i==k:
#             print(i)
            real_long_id.append(i)

right = 0
right_ratio=[]
false = 0
false_ratio=[]

date=[]
all_ratio=[]

account = 10000000
left = 0

buy = 0
sell = 0 
num = 0

result={}


print(real_long_id)
for i in real_long_id:
    for k in range(len(stock_data2)):
        if(i == str(stock_data2['id'][k]) and stock_data2['id'][k]!=20210617):
            if(stock_data2['등락률'][k] > 0):
                right+=1
                right_ratio.append(stock_data2['등락률'][k])
            else:
                false+=1
                false_ratio.append(stock_data2['등락률'][k])

            date.append(stock_data2['id'][k])
            all_ratio.append(stock_data2['등락률'][k])


            print("======================")
            print(stock_data2['id'][k])
            print("실제 다음날 등락률 : ",stock_data2['등락률'][k])
            buy = stock_data2['종가'][k]*(account//(stock_data2['종가'][k]* 1.00015))
            num = (account//(stock_data2['종가'][k]* 1.00015))
            print("구매 수:",num)
            # 다음날이 -1 임!! 
            sell = stock_data2['종가'][k-1]*num
            print("구매가격 : ",stock_data2['종가'][k] * 1.00015)
            print("구매가치 : ",buy)
            left = account - buy
            print("잔고--   : ",left)
            print("판매가격 : ",stock_data2['종가'][k-1])
            print("판매가치 : ",sell)

            account = sell * 0.9935 + left
            print("현재 자산 : ",account)

            print("현재 갯수 : ",(right + false))
            print("======================")



print(right_ratio)
print(false_ratio)
# print(all_check)
if(right + false !=0):
    print("acc = ",right/(right + false) * 100 ,"%")
print(" 최종 account : ",account)





<ipython-input-126-649c8db8bdf1>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_P_N['id'][i] = re.sub("\.","",news_P_N['id'][i])
<ipython-input-126-649c8db8bdf1>:35: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  long_id_news = news_P_N[news_P_N['num']>=7][news_P_N['ratio']>=16][news_P_N['id']>="20200710"].id


0      20200713
1      20200723
2      20200724
3      20200727
4      20200728
         ...   
99     20210603
100    20210607
101    20210611
102    20210615
103    20210616
Name: id, Length: 104, dtype: object
['20200713', '20200723', '20200724', '20200727', '20200728', '20200729', '20200730', '20200731', '20200804', '20200819', '20200820', '20200821', '20200824', '20200825', '20200901', '20200902', '20200904', '20200908', '20200909', '20200910', '20200911', '20200914', '20200917', '20200918', '20200921', '20200922', '20200923', '20200924', '20200925', '20200929', '20201005', '20201008', '20201012', '20201013', '20201014', '20201015', '20201016', '20201019', '20201021', '20201022', '20201023', '20201026', '20201027', '20201029', '20201105', '20201106', '20201111', '20201118', '20201119', '20201120', '20201123', '20201124', '20201125', '20201127', '20201130', '20201202', '20201203', '20201207', '20201209', '20201218', '20201221', '20201222', '20201223', '20201224', '20201228', '20201

20201012
실제 다음날 등락률 :  1.6
구매 수: 126.0
구매가격 :  75011.25
구매가치 :  9450000.0
잔고--   :  62085.82500000857
판매가격 :  76200
판매가치 :  9601200.0
현재 자산 :  9600878.02500001
현재 갯수 :  33
20201013
실제 다음날 등락률 :  -6.4304461942257225
구매 수: 125.0
구매가격 :  76211.43000000001
구매가치 :  9525000.0
잔고--   :  75878.02500000969
판매가격 :  71300
판매가치 :  8912500.0
현재 자산 :  8930446.77500001
현재 갯수 :  34
20201014
실제 다음날 등락률 :  -24.82468443197756
구매 수: 125.0
구매가격 :  71310.695
구매가치 :  8912500.0
잔고--   :  17946.775000009686
판매가격 :  53600
판매가치 :  6700000.0
현재 자산 :  6674396.77500001
현재 갯수 :  35
20201015
실제 다음날 등락률 :  -6.809701492537314
구매 수: 124.0
구매가격 :  53608.04000000001
구매가치 :  6646400.0
잔고--   :  27996.775000009686
판매가격 :  49950
판매가치 :  6193800.0
현재 자산 :  6181537.07500001
현재 갯수 :  36
20201016
실제 다음날 등락률 :  -0.4004004004004004
구매 수: 123.0
구매가격 :  49957.49250000001
구매가치 :  6143850.0
잔고--   :  37687.07500001043
판매가격 :  49750
판매가치 :  6119250.0
현재 자산 :  6117161.95000001
현재 갯수 :  37
20201019
실제 다음날 등락률 :  0.5025125628140703
구매 수: 

잔고--   :  49662.50000001956
판매가격 :  68300
판매가치 :  6898300.0
현재 자산 :  6903123.55000002
현재 갯수 :  68
20210108
실제 다음날 등락률 :  -7.6923076923076925
구매 수: 112.0
구매가격 :  61109.16500000001
구매가치 :  6843200.0
잔고--   :  59923.5500000203
판매가격 :  56400
판매가치 :  6316800.0
현재 자산 :  6335664.350000021
현재 갯수 :  69
20210113
실제 다음날 등락률 :  2.022058823529412
구매 수: 116.0
구매가격 :  54408.16
구매가치 :  6310400.0
잔고--   :  25264.350000021048
판매가격 :  55500
판매가치 :  6438000.0
현재 자산 :  6421417.350000021
현재 갯수 :  70
20210114
실제 다음날 등락률 :  -3.603603603603604
구매 수: 115.0
구매가격 :  55508.325000000004
구매가치 :  6382500.0
잔고--   :  38917.35000002105
판매가격 :  53500
판매가치 :  6152500.0
현재 자산 :  6151426.100000021
현재 갯수 :  71
20210115
실제 다음날 등락률 :  -6.5420560747663545
구매 수: 114.0
구매가격 :  53508.025
구매가치 :  6099000.0
잔고--   :  52426.10000002105
판매가격 :  50000
판매가치 :  5700000.0
현재 자산 :  5715376.100000021
현재 갯수 :  72
20210118
실제 다음날 등락률 :  2.2
구매 수: 114.0
구매가격 :  50007.50000000001
구매가치 :  5700000.0
잔고--   :  15376.100000021048
판매가격 :  51100
판매가

In [73]:
thre = [12,46,14,
        1,60,7,
        3,88,4,
        1,60,5,
        7,63,4,
        4,59,7,
        1,55,6,
        6,31,14,
        2,61,3,
        3,47,5,
        2,28,4,
        2,58,4,
        4,70,2,
        4,32,1,
        2,63,2,
        4,76,4,
        4,62,6,
       3,45,1,
       2,62,3,
       1,73,2,
       2,29,2,
       4,52,4,
       2,63,3,
       2,51,4,
       3,71,2,
       4,54,4,
       2,53,2,
       1,72,8,
       1,43,4,
       2,48,3,
       4,77,1,
       7,62,4,
       2,47,2,
       3,67,4,
       4,76,3,
       2,65,4,
       4,70,5,
       1,54,1,
       1,36,1,
       4,64,1,
       2,63,2,
       4,37,8,
       4,56,4,
       3,46,17,
       6,59,8,
       4,34,6,
       1,41,3,
       1,52,5,
       1,49,3,
       10,59,12,
       1,37,5,
       1,50,4,
       1,66,1,
       4,64,4,
       1,84,4,
       6,48,5,
       1,56,4,
       3,60,6,
       3,70,4,
       2,78,3,
       12,39,1,
       2,76,3,
       3,74,4,
       8,62,4,
       2,50,4,
       4,89,4,
       4,67,1,
       62,69,59,
       16,63,29,
       2,58,2,
       2,62,2,
       5,76,6,
       2,55,3,
       4,42,4,
       4,72,4,
       1,60,4,
       7,35,9,
       14,42,10,
       4,53,6,
       1,56,4,
       4,44,1,
       2,74,2,
       1,38,2,
       1,81,13,
       4,38,5,
       2,59,5,
       4,56,5,
       2,79,3,
       1,39,4]


len(thre)

267